# Hydrostatics

One consider a box with imposed pressure at the top, and initial zero velocity field. The purpose is to evaluate the ability of the numerical schemes to preserve the hydrostatic state.

The simulations are run on 2D and 3D FVCA meshes. The error is mesured on $\vec{v}$ and $P$.

In [ ]:
from trustutils import run

run.introduction('Y.G')
run.TRUST_parameters()

In [ ]:
run.useMEDCoupling()
import medcoupling as mc

def change_boundaries(med_path, D, poly):
    eps = 1e-6
    mcum = mc.ReadUMeshFromFile(med_path)
    if not poly: mcum.unPolyze()
    bb = mcum.getBoundingBox()

    mfum = mc.MEDFileUMesh.New()
    mfum.setMeshAtLevel(0, mcum)

    mf, _, _, _, _ = mcum.buildDescendingConnectivity()
    mfum.setMeshAtLevel(-1, mf)

    bf = mf.computeCellCenterOfMass()

    noms_cl = [["left", "right"]]
    if D == 3: noms_cl.append(["front", "back"])
    noms_cl.append(["down", "up"])
    for i in range(D):  # frontieres en x puis y (puis en z si extr)
        for j in range(2):  # min puis max
            g = []
            for idx, b in enumerate(bf):
                if abs(b[i] - bb[i][j]) < eps:
                    g.append(idx)
            grp = mc.DataArrayInt.New(g)
            grp.setName(noms_cl[i][j])
            mfum.addGroup(-1, grp)
    mfum.write(med_path, 2)

In [ ]:
from trustutils import run
from string import Template
import os, sys
run.useMEDCoupling()
import medcoupling as mc

run.reset()
m2d = {
    # 2D meshes
    "Cartesian":      ["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"],
    "Triangles":      ["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"],
    "Locally_Refined":["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"],
    "Quadrangles":    ["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"],
    "Kershaw2D":      ["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"],
    "Polygons":       ["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"],
}
m3d = {
    # 3D meshes
    "Tetra":          ["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"],
    "Hexa":           ["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"],
    "Prism":          ["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"],
    "Kershaw":        ["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"],
    "CheckerBoard":   ["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"],
    "PrismHexa":      ["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"],
    # "Random":         ["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"],
    "LocRaf":         ["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"],
    "Voronoi":        ["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"],
}
mxd = { 2 : m2d, 3 : m3d }
solvers = ["solveur_U_P", "implicite"]
meshes_dir = os.environ["TRUST_ROOT"] + "/Validation/Rapports_automatiques/Verification/PolyMAC/share/meshes"

# Trigger the copy of the src directory into the build directory:
run.initBuildDirectory()
origin, build_dir = os.getcwd(), run.BUILD_DIRECTORY
os.chdir(build_dir)

for s in solvers:
    for dim in [2, 3]:
        for m, dis in mxd[dim].items():
            for d in dis:
                os.system(f"mkdir -p {m}/{d}/{s}")
                with open(f"jdd_{dim}D.data", "r") as file: filedata = Template(file.read())
                result = filedata.substitute({"dis" : d, "solver" : s})
                with open(f"{m}/{d}/{s}/jdd.data", "w") as file: file.write(result)
                os.system(f"gunzip -c {meshes_dir}/{m}/jdd_1/mesh.med.gz >{m}/{d}/{s}/mesh.med")
                change_boundaries(f"{m}/{d}/{s}/mesh.med", dim, d.startswith("Poly"))
                run.addCase(f"{m}/{d}/{s}", f"jdd.data")

os.chdir(origin)

run.printCases()
run.runCases()
run.tablePerf()


In [ ]:
from trustutils import plot
import matplotlib.colors as mcolors
import pandas as pd
from matplotlib.lines import Line2D
c = list(mcolors.TABLEAU_COLORS.keys())
linestyle = {"PolyMAC_P0P1NC": ("solid", "o"), "VDF" : ("dotted", "*"), "PolyMAC_P0" : ("dashed", "v"), "VEFPreP1B" : ("dashdot", "s"), "PolyMAC" : ("dashed", "d")}

lines_and_markers = {dim : [Line2D([0], [0], color=c[k]) for k, _ in enumerate(dic.items())] for dim, dic in [(2, m2d), (3, m3d)]}
for i in [2, 3]: lines_and_markers[i] += [Line2D([0], [0], color="black", marker=m, linestyle=s) for _, (s, m) in linestyle.items()]
names_in_legend = {dim : list(dic.keys()) for dim, dic in [(2, m2d), (3, m3d)]}
for i in [2, 3]: names_in_legend[i] += list(linestyle.keys())

def make_custom_legend(graph, dim):
    try:
        for ax in graph.axs:
            ax.legend(lines_and_markers[dim], names_in_legend[dim], loc='upper center', ncol=4, bbox_to_anchor=(0.5, -0.1))
    except:
        graph.subplot.legend(lines_and_markers[dim], names_in_legend[dim], loc='upper center', ncol=4, bbox_to_anchor=(0.5, -0.1))

for s in solvers:
    a = plot.Graph(f"Residuals - {s}")
    for dim, dic in [(2, m2d), (3, m3d)]:
        for k, (m, dis) in enumerate(dic.items()):
            for d in dis:
                a.addResidu(f"{m}/{d}/{s}/jdd.dt_ev", label=None, color=c[k], linestyle=linestyle[d][0], marker=linestyle[d][1], markersize=12)

    a.scale(yscale='log')
make_custom_legend(a, dim)

In [ ]:
import pandas as pd
def highlighter(cell_value):
    if float(cell_value) > 1e-6:
        return "background-color: red"

max_errorV = {s : {} for s in solvers}
max_errorP = {s : {} for s in solvers}

for s in solvers:
    for dim, dic in [(2, m2d), (3, m3d)]:
        for k, (m, dis) in enumerate(dic.items()):
            for d in dis:
                errV = plot.loadText(f"{m}/{d}/{s}/jdd_ERRV.son")
                errP = plot.loadText(f"{m}/{d}/{s}/jdd_ERRP.son")
                if m in max_errorV[s]:
                    max_errorV[s][m][d] = errV[-1][-1]
                else:
                    max_errorV[s][m] = {d : errV[-1][-1]}
                if m in max_errorP[s]:
                    max_errorP[s][m][d] = errP[-1][-1]
                else:
                    max_errorP[s][m] = {d : errP[-1][-1]}    

In [ ]:
df = pd.DataFrame.from_dict(max_errorV["solveur_U_P"]).T
df.style.format("{:.2e}".format).applymap(highlighter).set_caption("Max error on velocity - solveur_U_P")

In [ ]:
df = pd.DataFrame.from_dict(max_errorV["implicite"]).T
df.style.format("{:.2e}".format).applymap(highlighter).set_caption("Max error on velocity - implicite")

In [ ]:
df = pd.DataFrame.from_dict(max_errorP["solveur_U_P"]).T
df.style.format("{:.2e}".format).applymap(highlighter).set_caption("Max error on pressure - solveur_U_P")

In [ ]:
df = pd.DataFrame.from_dict(max_errorP["implicite"]).T
df.style.format("{:.2e}".format).applymap(highlighter).set_caption("Max error on pressure - implicite")